In [1]:
x=1
x**2

1

This is an example from simpy.

In [1]:
"""
Bank renege example

Covers:

- Resources: Resource
- Condition events

Scenario:
  A counter with a random service time and customers who renege. Based on the
  program bank08.py from TheBank tutorial of SimPy 2. (KGM)

"""

import random

import simpy

RANDOM_SEED = 42
NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience


def source(env, number, interval, counter):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, f'Customer{i:02d}', counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer(env, name, counter, time_in_bank):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    print(f'{arrive:7.4f} {name}: Here I am')

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # We got to the counter
            print(f'{env.now:7.4f} {name}: Waited {wait:6.3f}')

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print(f'{env.now:7.4f} {name}: Finished')

        else:
            # We reneged
            print(f'{env.now:7.4f} {name}: RENEGED after {wait:6.3f}')


# Setup and start the simulation
print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Start processes and run
counter = simpy.Resource(env, capacity=1)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run()

Bank renege
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 3.8595 Customer00: Finished
10.2006 Customer01: Here I am
10.2006 Customer01: Waited  0.000
12.7265 Customer02: Here I am
13.9003 Customer02: RENEGED after  1.174
23.7507 Customer01: Finished
34.9993 Customer03: Here I am
34.9993 Customer03: Waited  0.000
37.9599 Customer03: Finished
40.4798 Customer04: Here I am
40.4798 Customer04: Waited  0.000
43.1401 Customer04: Finished


The following is from ChatGPT.

In [11]:
import simpy
import random

# Customer class to model the customers arriving at the system
class Customer:
    def __init__(self, arrival_time):
        self.arrival_time = arrival_time
        self.wait_time = 0
        self.service_time = 0
        self.system_time = 0

def customer_arrivals(env, servers, sim_duration):
    """Handles the arrival of customers."""
    customer_id = 0
    while env.now < sim_duration:
        customer_id += 1
        arrival_time = env.now
        customer = Customer(arrival_time)
        print(f"Time {env.now:.2f}: Customer {customer_id} arrives.")
        env.process(customer_service(env, customer, servers, customer_id))
        yield env.timeout(random.expovariate(l))  # Exponential inter-arrival time

def customer_service(env, customer, servers, customer_id):
    """Simulates the service of a customer."""
    # Request a server
    with servers.request() as request:
        print(f"Time {env.now:.2f}: Customer {customer_id} waits for a server.")
        yield request  # Wait for a free server
        
        # Customer starts service
        wait_time = env.now - customer.arrival_time
        customer.wait_time = wait_time
        service_time = random.expovariate(u)  # Exponential service time
        customer.service_time = service_time
        print(f"Time {env.now:.2f}: Customer {customer_id} starts service. Wait time = {wait_time:.2f}, Service time = {service_time:.2f}")

        # Simulate service time
        yield env.timeout(service_time)
        
        # Customer completes service
        customer.system_time = env.now - customer.arrival_time
        print(f"Time {env.now:.2f}: Customer {customer_id} finishes service. Total time in system = {customer.system_time:.2f}")

        # Store results
        waiting_times.append(customer.wait_time)
        system_times.append(customer.system_time)


In [13]:
# Parameters
l = 5  # Rate of customer arrivals (lambda)
u = 1  # Rate of service (mu)
m = 2  # Number of servers
sim_duration = 10  # Simulation duration (in time units)

# Lists to store results
waiting_times = []
system_times = []


# Initialize the environment and server pool
env = simpy.Environment()
servers = simpy.Resource(env, capacity=m)

# Start the arrival process
env.process(customer_arrivals(env, servers, sim_duration))

# Run the simulation
env.run()

# Output the results
print(f"\nAverage waiting time: {sum(waiting_times) / len(waiting_times):.2f}")
print(f"Average system time: {sum(system_times) / len(system_times):.2f}")
print(f"Total customers served: {len(waiting_times)}")

Time 0.00: Customer 1 arrives.
Time 0.00: Customer 1 waits for a server.
Time 0.00: Customer 1 starts service. Wait time = 0.00, Service time = 1.77
Time 0.49: Customer 2 arrives.
Time 0.49: Customer 2 waits for a server.
Time 0.49: Customer 2 starts service. Wait time = 0.00, Service time = 1.74
Time 0.55: Customer 3 arrives.
Time 0.55: Customer 3 waits for a server.
Time 0.68: Customer 4 arrives.
Time 0.68: Customer 4 waits for a server.
Time 1.01: Customer 5 arrives.
Time 1.01: Customer 5 waits for a server.
Time 1.28: Customer 6 arrives.
Time 1.28: Customer 6 waits for a server.
Time 1.37: Customer 7 arrives.
Time 1.37: Customer 7 waits for a server.
Time 1.39: Customer 8 arrives.
Time 1.39: Customer 8 waits for a server.
Time 1.77: Customer 1 finishes service. Total time in system = 1.77
Time 1.77: Customer 3 starts service. Wait time = 1.23, Service time = 0.15
Time 1.93: Customer 3 finishes service. Total time in system = 1.38
Time 1.93: Customer 4 starts service. Wait time = 1.